**Ressources**:

- [TF Tutorial](https://www.tensorflow.org/tutorials/images/data_augmentation)
- [Image Augmentation](https://towardsdatascience.com/image-augmentation-for-deep-learning-using-keras-and-histogram-equalization-9329f6ae5085)